# GeoParquet Konverteringsverktøy

Dette verktøyet konverterer geografiske datafiler til GeoParquet-format.

## Hvordan bruke dette verktøyet

1. Last opp dine geografiske datafiler til mappen `data/raw`
2. Kjør alle cellene i denne notebooken
3. De konverterte filene vil bli lagret i mappen `data/processed`

## Støttede filformater

- Parquet-filer med geografisk informasjon


## Last inn nødvendige pakker

Kjør cellen nedenfor for å importere pakkene som verktøyet trenger:

In [ ]:
!pip install -r requirements.txt

In [2]:
import os
import pandas as pd
import geopandas as gpd
from shapely import wkb
import time


## Definer funksjoner
Kjør cellen nedenfor for å definere funksjonene som konverterer filene:

In [3]:
def opprett_mappestruktur(data_mappe="./data"):
    """
    Oppretter nødvendig mappestruktur for konvertering.

    Args:
        data_mappe: Sti til hovedmappen for data

    Returns:
        dict: Stier til opprettede mapper
    """
    # Definer mappestruktur
    rå_mappe = os.path.join(data_mappe, "raw")
    prosessert_mappe = os.path.join(data_mappe, "processed")

    # Opprett mapper
    os.makedirs(data_mappe, exist_ok=True)
    os.makedirs(rå_mappe, exist_ok=True)
    os.makedirs(prosessert_mappe, exist_ok=True)

    print(f"Mappestruktur opprettet.")

    # Returner stier for senere bruk
    return {
        "data_mappe": os.path.abspath(data_mappe),
        "rå_mappe": os.path.abspath(rå_mappe),
        "prosessert_mappe": os.path.abspath(prosessert_mappe)
    }

def legg_til_tidspartisjoneringskolonner(df, tid_kolonne='date_time_utc'):
    """
    Legger til kolonner for tidspartisjonering (år, måned, dag, time).
    """
    if tid_kolonne not in df.columns:
        print(f"Advarsel: Tidsstempelkolonne '{tid_kolonne}' finnes ikke")
        return df

    if not pd.api.types.is_datetime64_any_dtype(df[tid_kolonne]):
        print(f"Advarsel: Kolonnen '{tid_kolonne}' er ikke en datetime-kolonne")
        return df

    df_med_tid = df.copy()
    df_med_tid['year'] = df_med_tid[tid_kolonne].dt.year
    df_med_tid['month'] = df_med_tid[tid_kolonne].dt.month
    df_med_tid['day'] = df_med_tid[tid_kolonne].dt.day
    # df_med_tid['hour'] = df_med_tid[tid_kolonne].dt.hour

    return df_med_tid

def opprett_geodataframe(df):
    """
    Oppretter en GeoDataFrame fra en DataFrame ved å finne koordinater eller geometrikolonner.
    """
    from shapely import wkt
    # Sjekk for lat/long kolonner
    if 'longitude' in df.columns and 'latitude' in df.columns:
        try:
            return gpd.GeoDataFrame(
                df,
                geometry=gpd.points_from_xy(df.longitude, df.latitude),
                crs="EPSG:4326"
            )
        except Exception as e:
            print(f"Kunne ikke opprette geometri fra lat/long: {e}")

    # Sjekk for andre geometrikolonner
    geom_kolonner = [col for col in df.columns if any(
        term in col.lower() for term in ['geom', 'coord', 'point', 'polygon', 'linestring', 'wkt']
    )]

    for col in geom_kolonner:
        if df[col].dtype != 'object':
            continue

        try:
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
        except Exception:
            continue

    return None

def lagre_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner):
    """
    Lagrer en GeoDataFrame som partisjonert GeoParquet.
    """
    from shapely import wkb # NB!! Ikke fjern!

    # Konverterer geometri til WKB for å kunne partisjonere
    df_med_wkb = gdf.copy()
    df_med_wkb['geometry_wkb'] = df_med_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    df_for_partisjon = df_med_wkb.drop(columns=['geometry'])

    # Utfør partisjonering
    df_for_partisjon.to_parquet(målfilsti, partition_cols=partisjon_kolonner)

    # Konverter hver partisjonert fil tilbake til GeoParquet
    konverter_partisjonerte_filer_til_geoparquet(målfilsti)
    return målfilsti

def opprett_år_mappe(år, målfilsti):
    """Oppretter en mappe for et spesifikt år."""
    år_mappe = os.path.join(målfilsti, f"year={år}")
    os.makedirs(år_mappe, exist_ok=True)
    return år_mappe

def forbered_wkb_dataframe(år_df):
    """Forbereder en dataframe med WKB-konvertert geometri for partisjonering."""
    # Konverter geometri til WKB for å kunne partisjonere
    år_df_wkb = år_df.copy()
    år_df_wkb['geometry_wkb'] = år_df_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    år_df_wkb = år_df_wkb.drop(columns=['geometry'])
    return år_df_wkb

def konverter_fil_til_geoparquet(src_file, dst_file):
    """Konverterer en enkelt fil fra WKB-format til GeoParquet."""
    try:
        part_df = pd.read_parquet(src_file)
        if 'geometry_wkb' not in part_df.columns:
            return False

        part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
        part_df = part_df.drop(columns=['geometry_wkb'])
        part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

        # Sørg for at målmappen eksisterer
        os.makedirs(os.path.dirname(dst_file), exist_ok=True)
        part_gdf.to_parquet(dst_file)
        return True
    except Exception as e:
        print(f"Advarsel: Kunne ikke konvertere {src_file} til GeoParquet: {e}")
        return False

def kopier_og_konverter_filer(temp_år_mappe, år_mappe):
    """Kopierer og konverterer filer fra temp-mappen til målmappen."""
    suksess_count = 0
    feil_count = 0

    # Opprett først alle mappene
    for root, dirs, _ in os.walk(temp_år_mappe):
        for directory in dirs:
            src_dir = os.path.join(root, directory)
            rel_path = os.path.relpath(src_dir, temp_år_mappe)
            dst_dir = os.path.join(år_mappe, rel_path)
            os.makedirs(dst_dir, exist_ok=True)

    # Kopier og konverter filene
    for root, _, files in os.walk(temp_år_mappe):
        for file in files:
            src_file = os.path.join(root, file)
            rel_path = os.path.relpath(src_file, temp_år_mappe)
            dst_file = os.path.join(år_mappe, rel_path)

            if konverter_fil_til_geoparquet(src_file, dst_file):
                suksess_count += 1
            else:
                feil_count += 1

    return suksess_count, feil_count

def lagre_hierarkisk_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner):
    """
    Lagrer en GeoDataFrame med hierarkisk partisjonering der år kommer først.

    Args:
        gdf: GeoDataFrame som skal lagres
        målfilsti: Base målsti
        partisjon_kolonner: Liste av kolonner for partisjonering etter år

    Returns:
        str: Målfilsti hvis vellykket
    """
    # Sjekk om year er en av kolonnene
    if 'year' not in gdf.columns:
        print("Advarsel: 'year' kolonne finnes ikke for hierarkisk partisjonering")
        return lagre_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner)

    # Opprett en temporær mappe for midlertidig lagring
    import tempfile
    temp_dir = tempfile.mkdtemp()

    try:
        # Partisjonér på år
        år_grupper = gdf.groupby('year')

        total_suksess = 0
        total_feil = 0

        # Filtrer ut year fra partisjoneringskolonnene
        andre_kolonner = [col for col in partisjon_kolonner if col != 'year']

        # Behandle hvert år
        for år, år_df in år_grupper:
            år_mappe = opprett_år_mappe(år, målfilsti)

            # Hvis det er andre partisjoneringskolonner, partisjonér videre
            if andre_kolonner:
                # Forbered dataframe for partisjonering
                år_df_wkb = forbered_wkb_dataframe(år_df)

                # Lag temp-mappe for dette året
                temp_år_mappe = os.path.join(temp_dir, f"year={år}")
                os.makedirs(temp_år_mappe, exist_ok=True)

                # Partisjonér data til temp-mappen
                år_df_wkb.to_parquet(temp_år_mappe, partition_cols=andre_kolonner)

                # Kopier og konverter filer til endelig plassering
                suksess, feil = kopier_og_konverter_filer(temp_år_mappe, år_mappe)
                total_suksess += suksess
                total_feil += feil
            else:
                # Hvis ingen andre partisjoneringskolonner, lagre direkte
                år_df.to_parquet(os.path.join(år_mappe, "data.parquet"))
                total_suksess += 1

        if total_feil > 0:
            print(f"Info: {total_suksess} filer konvertert, {total_feil} filer med feil")
    except Exception as e:
        print(f"Feil under hierarkisk partisjonering: {e}")
        return False
    finally:
        # Rydd opp temp-mappen
        import shutil
        shutil.rmtree(temp_dir)

    return målfilsti

def konverter_partisjonerte_filer_til_geoparquet(rotmappe):
    """
    Konverterer alle partisjonerte parquet-filer til GeoParquet format.
    """
    from shapely import wkb # NB!! Ikke fjern!
    feil_count = 0

    for root, _, files in os.walk(rotmappe):
        for file in files:
            if not file.endswith('.parquet'):
                continue

            parquet_path = os.path.join(root, file)
            try:
                # Les dataframe
                part_df = pd.read_parquet(parquet_path)

                # Hopp over hvis den ikke har geometry_wkb
                if 'geometry_wkb' not in part_df.columns:
                    continue

                # Konverter WKB tilbake til geometri
                part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
                part_df = part_df.drop(columns=['geometry_wkb'])

                # Lag GeoDataFrame
                part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

                # Skriv GeoParquet-filen
                part_gdf.to_parquet(parquet_path)
            except Exception as e:
                print(f"Feil ved konvertering av {parquet_path}: {e}")
                feil_count += 1

    if feil_count > 0:
        print(f"Advarsel: {feil_count} filer kunne ikke konverteres til GeoParquet")

def konverter_parquet_til_geoparquet(filsti, målfilsti, partisjon_kolonner=None, hierarkisk_år=True):
    """
    Konverterer parquet-fil til GeoParquet-format med partisjonering.

    Args:
        filsti: Sti til parquet-filen
        målfilsti: Sti hvor GeoParquet-filen skal lagres
        partisjon_kolonner: Liste av kolonnenavn som skal brukes for partisjonering
        hierarkisk_år: Hvis True, partisjoneres hierarkisk med år først
    """
    try:
        df = pd.read_parquet(filsti)
    except Exception as e:
        print(f"Kunne ikke lese parquet-fil: {e}")
        return False

    # Håndter tidspartisjonering
    tidspartisjonering = False
    if partisjon_kolonner and 'date_time_utc' in partisjon_kolonner:
        partisjon_kolonner.remove('date_time_utc')
        tidspartisjonering = True

    # Legg til tidspartisjoneringskolonner
    if tidspartisjonering:
        df = legg_til_tidspartisjoneringskolonner(df)
        if hierarkisk_år:
            # For hierarkisk partisjonering, year håndteres separat
            # Legg til andre tidskolonner til partisjoneringskolonnene
            tidskolonner = ['month', 'day']
            partisjon_kolonner = tidskolonner + (partisjon_kolonner or [])
        else:
            # Standard flat partisjonering
            partisjon_kolonner = ['year', 'month', 'day'] + (partisjon_kolonner or [])

    # Opprett GeoDataFrame
    gdf = opprett_geodataframe(df)
    if gdf is None:
        return False

    # Sjekk at alle partisjoneringskolonner finnes
    if partisjon_kolonner and not all(col in gdf.columns for col in partisjon_kolonner):
        print(f"Advarsel: Ikke alle partisjoneringskolonner finnes i datasettet")
        manglende = [col for col in partisjon_kolonner if col not in gdf.columns]
        print(f"Manglende kolonner: {manglende}")
        return False

    # Lagre med eller uten partisjonering
    if partisjon_kolonner:
        if hierarkisk_år and tidspartisjonering:
            return lagre_hierarkisk_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner)
        else:
            return lagre_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner)
    else:
        gdf.to_parquet(målfilsti)
        return målfilsti

def konverter_alle_parquet_filer(data_mappe, partisjon_kolonner=None, hierarkisk_år=True):
    """
    Konverterer alle parquet-filer i en mappe til GeoParquet.
    """
    rå_mappe = os.path.join(data_mappe, "raw")
    prosessert_mappe = os.path.join(data_mappe, "processed")

    # Opprett mapper
    os.makedirs(rå_mappe, exist_ok=True)
    os.makedirs(prosessert_mappe, exist_ok=True)

    resultater = {
        "konvertert": [],
        "feilet": []
    }

    # Finn alle parquet-filer
    parquet_filer = [f for f in os.listdir(rå_mappe)
                     if f.lower().endswith('.parquet') and os.path.isfile(os.path.join(rå_mappe, f))]

    if not parquet_filer:
        print("Ingen parquet-filer funnet i råmappen")
        return resultater

    # Konverter hver fil
    for filnavn in parquet_filer:
        filsti = os.path.join(rå_mappe, filnavn)
        base_filnavn = os.path.splitext(filnavn)[0]
        målfilsti = os.path.join(prosessert_mappe, f"{base_filnavn}.parquet")

        # Konverter filen
        resultat = konverter_parquet_til_geoparquet(
            filsti,
            målfilsti,
            partisjon_kolonner.copy() if partisjon_kolonner else None,
            hierarkisk_år
        )

        if resultat:
            resultater["konvertert"].append(filsti)
        else:
            resultater["feilet"].append(filsti)
            print(f"Kunne ikke konvertere: {filsti}")

    return resultater

def vis_partisjoneringsstruktur(konvertert_sti):
    """
    Viser partisjoneringsstrukturen for en konvertert fil.
    """

    for root, dirs, files in os.walk(konvertert_sti, topdown=True, followlinks=False):
        nivå = root.replace(konvertert_sti, "").count(os.sep)
        innrykk = "    " * (nivå + 1)

        # Vis mappenavnet
        mappe_navn = os.path.basename(root)
        if mappe_navn:  # Ikke vis for rot-mappen
            print(f"{innrykk}- {mappe_navn}")

        # Vis antall filer i dypeste mapper
        if not dirs and files:
            print(f"{innrykk}  Inneholder {len(files)} filer")

def vis_datasett_info(filsti):
    """
    Viser informasjon om et GeoParquet datasett.
    """
    try:
        gdf = gpd.read_parquet(filsti)
        print(f"  Datasett størrelse: {len(gdf)} rader, {len(gdf.columns)} kolonner")
        return gdf
    except Exception as e:
        print(f"  Kunne ikke lese filen: {e}")
        return None

def start_konvertering(data_mappe="./data", partisjon_kolonner=None, hierarkisk_år=True):
    """
    Start konverteringsprosessen fra Parquet til GeoParquet.

    Args:
        data_mappe: Sti til datamappen
        partisjon_kolonner: Liste av kolonner for partisjonering
        hierarkisk_år: Hvis True, partisjoneres først på år, så andre kolonner
    """
    start_tid = time.time()
    print(f"Starter konvertering av parquet-filer i {data_mappe} ...\n")

    # Opprett raw-mappen
    rå_mappe = os.path.join(data_mappe, "raw")
    os.makedirs(rå_mappe, exist_ok=True)

    # Utfør konverteringen
    resultater = konverter_alle_parquet_filer(data_mappe, partisjon_kolonner, hierarkisk_år)

    print("Konvertering og partisjonering utført:")
    print(f"• Total behandlingstid: {time.time() - start_tid:.2f} sekunder")
    print(f"• Konverterte filer: {len(resultater['konvertert'])}")

    return resultater

## Opprett mappestruktur

Kjør cellen nedenfor for å opprette nødvendige mapper:

In [4]:
mappestier = opprett_mappestruktur()

Mappestruktur opprettet.


## Start konvertering og partisjonering

Nå er alt klart! Kjør cellen nedenfor for å starte konvertering og partisjonering:

In [5]:
# Kun konvertering
# resultater = start_konvertering(data_mappe="./data")

# Manuell konvertering og partisjonering
resultater = start_konvertering(data_mappe="./data", partisjon_kolonner=["date_time_utc", "ship_type", "ship_name"])


Starter konvertering av parquet-filer i ./data...

Konvertering og partisjonering utført:
• Total behandlingstid: 10.15 sekunder
• Konverterte filer: 10
